In [5]:
import sys
from pathlib import Path

from rich.jupyter import display

ROOT = Path().resolve().parent
sys.path.append(str(ROOT))


In [6]:
import pandas as pd
from sklearn.metrics import confusion_matrix

from research.urgency import UrgencyPredictorNLP
from research.scarcity import ScarcityPredictorNLP
urgency_dataset = pd.read_csv("datasets/urgency.csv")


urgency_predictor  = UrgencyPredictorNLP()


# Predict only giving the text column
y_true = urgency_dataset["is_urgency"]
y_pred = urgency_predictor.predict_multiple(urgency_dataset["text"])

confusion_matrix(y_true, y_pred)

array([[11,  0],
       [ 0, 25]], dtype=int64)

In [7]:

scarcity_dataset = pd.read_csv("datasets/scarcity.csv" )
scarcity_predictor = ScarcityPredictorNLP()
y_true = scarcity_dataset["is_scarcity"]
y_pred = scarcity_predictor.predict_multiple(scarcity_dataset["text"])
confusion_matrix(y_true, y_pred)

array([[2, 0],
       [0, 7]], dtype=int64)

In [16]:
# Read from the datasets/images directory, get all the name of files, count scarcity and urgency based on if it's in the name

import os
from collections import Counter

directory_path = "datasets/images"

file_metrics = Counter({'scarcity': 0, 'urgency': 0, 'other': 0})

if os.path.exists(directory_path):
    files = [f.lower() for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

    for filename in files:
        if "scarcity" in filename:
            file_metrics['scarcity'] += 1
        elif "urgency" in filename:
            file_metrics['urgency'] += 1
        else :
            file_metrics['other'] += 1
    print(f"Total Files Scanned: {len(files)}")
    print(f"Scarcity Count: {file_metrics['scarcity']}")
    print(f"Urgency Count: {file_metrics['urgency']}")
else:
    print(f"Error: The directory '{directory_path}' was not found.")

Total Files Scanned: 73
Scarcity Count: 30
Urgency Count: 43
